## ノック41: データを読み込んで利用データを整形

In [1]:
# データの読み込み
import pandas as pd
customer = pd.read_csv("customer_join.csv")
uselog_months = pd.read_csv("use_log_months.csv")

In [2]:
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [3]:
uselog_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [4]:
# 今回は当月と過去1ヶ月分のデータのみで予測を行う。まずは必要なデータを作成
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()

print(year_months)
for i in range(1, len(year_months)):
  tmp = uselog_months.loc[uselog_months["年月"] == year_months[i]]
  tmp.rename(columns={"count":"count_0"}, inplace=True)
  tmp_before = uselog_months.loc[uselog_months["年月"] == year_months[i-1]]
  del tmp_before["年月"]
  tmp_before.rename(columns={"count":"count_1"}, inplace=True)
  tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
  uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

[201804, 201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812, 201901, 201902, 201903]


C:\Users\nao23\AppData\Local\Temp\ipykernel_9512\704579143.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.rename(columns={"count":"count_0"}, inplace=True)
C:\Users\nao23\AppData\Local\Temp\ipykernel_9512\704579143.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_before.rename(columns={"count":"count_1"}, inplace=True)
C:\Users\nao23\AppData\Local\Temp\ipykernel_9512\704579143.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


## ノック42: 退会前月の退会顧客データを作成

In [5]:
# end_date列の1ヶ月前の年月を取得し、uselogとcustomer_id, 年月をキーにして結合する
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"] == 1]

exit_customer["exit_date"] = None
exit_customer["exit_date"] = pd.to_datetime(exit_customer["exit_date"])
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])

for i in range(len(exit_customer)):
  exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)

exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

C:\Users\nao23\AppData\Local\Temp\ipykernel_9512\1284317935.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["exit_date"] = None
C:\Users\nao23\AppData\Local\Temp\ipykernel_9512\1284317935.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["exit_date"] = pd.to_datetime(exit_customer["exit_date"])
C:\Users\nao23\AppData\Local\Temp\ipykernel_9512\1284317935.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

33851


C:\Users\nao23\AppData\Local\Temp\ipykernel_9512\1284317935.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
C:\Users\nao23\AppData\Local\Temp\ipykernel_9512\1284317935.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
C:\Users\nao23\AppData\Local\Temp\ipykernel_9512\1284317935.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [6]:
# 退会した顧客の退会前日のデータのみなので、欠損値が多い。
# 欠損値を削除する処理

# customerデータにはname列以外に欠損値は無いため、
# 逆に言えばname列に欠損値がある場合は退会前月データと結合できない不要データであるため削除
exit_uselog = exit_uselog.dropna(subset=["name"])

# 削除結果が一致している事を確認
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))

exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


## ノック43: 継続顧客のデータを作成

In [7]:
# 継続顧客を絞り込んでuselogデータに結合する
# 継続顧客はどの月のデータを用いても良い
conti_customer = customer.loc[customer["is_deleted"] == 0]
conti_uselog = pd.merge(uselog, conti_customer, on="customer_id", how="left")
print(len(conti_uselog))

# nameが欠損しているデータを削除
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [8]:
# 退会顧客のデータ件数が継続顧客に比して少ないので、
# 継続顧客のデータをランダムサンプリングかつ重複削除して、データ全体をアンダーサンプリングする
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201807,PL117922,2,5.0,XXXX,C02,F,2015-07-01,NaN,CA1,...,デイタイム,7500.0,通常,3.916667,3.0,7.0,1.0,1.0,2019-04-30,45.0
1,201812,OA388131,2,7.0,XXXX,C02,F,2016-01-01,NaN,CA1,...,デイタイム,7500.0,通常,3.750000,3.5,7.0,1.0,1.0,2019-04-30,39.0
2,201807,TS073726,6,6.0,XXXXX,C02,M,2017-05-01,NaN,CA1,...,デイタイム,7500.0,通常,5.833333,6.0,7.0,4.0,1.0,2019-04-30,23.0
3,201901,GD212992,4,5.0,XXXX,C03,F,2015-06-01,NaN,CA1,...,ナイト,6000.0,通常,5.250000,5.0,9.0,3.0,1.0,2019-04-30,46.0
4,201809,TS369884,9,6.0,XXXX,C03,M,2018-07-05,NaN,CA1,...,ナイト,6000.0,通常,8.111111,8.0,10.0,6.0,1.0,2019-04-30,9.0


In [9]:
# 継続顧客データと退会顧客データを縦に結合する
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201807,PL117922,2,5.0,XXXX,C02,F,2015-07-01,NaT,CA1,...,7500.0,通常,3.916667,3.0,7.0,1.0,1.0,2019-04-30,45.0,NaT
1,201812,OA388131,2,7.0,XXXX,C02,F,2016-01-01,NaT,CA1,...,7500.0,通常,3.750000,3.5,7.0,1.0,1.0,2019-04-30,39.0,NaT
2,201807,TS073726,6,6.0,XXXXX,C02,M,2017-05-01,NaT,CA1,...,7500.0,通常,5.833333,6.0,7.0,4.0,1.0,2019-04-30,23.0,NaT
3,201901,GD212992,4,5.0,XXXX,C03,F,2015-06-01,NaT,CA1,...,6000.0,通常,5.250000,5.0,9.0,3.0,1.0,2019-04-30,46.0,NaT
4,201809,TS369884,9,6.0,XXXX,C03,M,2018-07-05,NaT,CA1,...,6000.0,通常,8.111111,8.0,10.0,6.0,1.0,2019-04-30,9.0,NaT


## ノック44: 予測する月の在籍期間を作成

In [10]:
# 在籍期間の列を作成
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
  delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
  predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

C:\Users\nao23\AppData\Local\Temp\ipykernel_9512\928780495.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data["period"][i] = int(delta.years*12 + delta.months)
C:\Users\nao23\AppData\Local\Temp\ipykernel_9512\928780495.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data["period"][i] = int(delta.years*12 + delta.months)
C:\Users\nao23\AppData\Local\Temp\ipykernel_9512\928780495.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201807,PL117922,2,5.0,XXXX,C02,F,2015-07-01,NaT,CA1,...,3.916667,3.0,7.0,1.0,1.0,2019-04-30,45.0,NaT,36,2018-07-01
1,201812,OA388131,2,7.0,XXXX,C02,F,2016-01-01,NaT,CA1,...,3.750000,3.5,7.0,1.0,1.0,2019-04-30,39.0,NaT,35,2018-12-01
2,201807,TS073726,6,6.0,XXXXX,C02,M,2017-05-01,NaT,CA1,...,5.833333,6.0,7.0,4.0,1.0,2019-04-30,23.0,NaT,14,2018-07-01
3,201901,GD212992,4,5.0,XXXX,C03,F,2015-06-01,NaT,CA1,...,5.250000,5.0,9.0,3.0,1.0,2019-04-30,46.0,NaT,43,2019-01-01
4,201809,TS369884,9,6.0,XXXX,C03,M,2018-07-05,NaT,CA1,...,8.111111,8.0,10.0,6.0,1.0,2019-04-30,9.0,NaT,1,2018-09-01


## ノック45: 欠損値を除去

In [11]:
# 欠損値の数を把握
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               261
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [12]:
# 継続顧客は除外して退会顧客のみ対象とするため、count_1を持つ顧客は除外する
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2633
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2633
period                  0
now_date                0
dtype: int64

## ノック46: 文字列型の変数を処理できるように整形

In [13]:
# 今回の予測に用いるデータに絞り込み
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
print(len(predict_data))
predict_data.head()

3685


,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,デイタイム,F,5.0,1.0,36,0.0
1,通常,デイタイム,F,7.0,1.0,35,0.0
2,通常,デイタイム,M,6.0,1.0,14,0.0
3,通常,ナイト,F,5.0,1.0,43,0.0
4,通常,ナイト,M,6.0,1.0,1,0.0


In [14]:
# カテゴリカル変数を用いてダミー変数を作成
predict_data = pd.get_dummies(predict_data)  #一括でダミー変数化する処理
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1.0,36,0.0,False,False,True,False,True,False,True,False
1,7.0,1.0,35,0.0,False,False,True,False,True,False,True,False
2,6.0,1.0,14,0.0,False,False,True,False,True,False,False,True
3,5.0,1.0,43,0.0,False,False,True,False,False,True,True,False
4,6.0,1.0,1,0.0,False,False,True,False,False,True,False,True


In [15]:
# 余分なダミー変数を除去
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,5.0,1.0,36,0.0,False,False,False,True,True
1,7.0,1.0,35,0.0,False,False,False,True,True
2,6.0,1.0,14,0.0,False,False,False,True,False
3,5.0,1.0,43,0.0,False,False,False,False,True
4,6.0,1.0,1,0.0,False,False,False,False,False


## ノック47: 決定木を用いて退会予測モデルを作成

In [16]:
# 決定木アルゴリズムで分析
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection  #学習データと評価データの分割に必要なライブラリ

# 退会と継続のデータ件数を一緒になるようアンダーサンプリングする処理をする(2行目)
exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

X = pd.concat([exit,conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)  # 0は継続,1は退会を示す

[0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0.
 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0.
 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0.
 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1.
 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1.
 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0.
 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0.
 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0.

In [17]:
# 実際の値y_test と一緒のデータフレームに格納する
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred})
print(len(results_test))
results_test.head()

526


,y_test,y_pred
1806,0.0,0.0
339,1.0,1.0
1775,0.0,0.0
1468,0.0,0.0
1162,0.0,0.0


## ノック48: 予測モデルの評価を行い、モデルのチューニングを行う

In [18]:
# モデルの正解率を確認する
correct = len(results_test.loc[results_test["y_test"] == results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.8745247148288974


In [19]:
# 教師データと評価データを用いてモデルの精度を確認する
print(model.score(X_train, y_train))
print(model.score(X_test, y_test)) # 今回の結果は教師データに適合しすぎているため過学習の傾向がある

0.9847908745247148
0.8745247148288974


In [20]:
# 決定木モデルを表示する
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

# 決定木の分岐深さを調整する事で、モデルの簡易度を調整できる。
# 今回は、調整すると評価データの精度が向上する結果となった。
model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.9258555133079848
0.903041825095057


## ノック49: モデルに奇与している変数を確認する

In [21]:
importance = pd.DataFrame({"feature_name":X.columns, "coefficient":model.feature_importances_})
importance

,feature_name,coefficient
0,count_1,0.351868
1,routine_flg,0.120934
2,period,0.525084
3,campaign_name_入会費半額,0.000000
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000000
6,class_name_デイタイム,0.002114
7,gender_F,0.000000


## ノック50: 顧客の退会を予測

In [22]:
# 予測モデルを作成する
# まずは一例の変数を指定してみる
count_1 = 3
routine_flg = 1
period = 10
campaign_name = "入会費無料"
class_name = "オールタイム"
gender = "M"

# データ加工
if campaign_name == "入会費半額":
  campaign_name_list = [1,0]
elif campaign_name == "入会費無料":
  campaign_name_list = [0,1]
elif campaign_name == "通常":
  campaign_name_list = [0,0]
if class_name == "オールタイム":
  class_name_list = [1,0]
elif class_name == "デイタイム":
  class_name_list = [0,1]
elif class_name == "ナイト":
  class_name_list = [0,0]
if gender == "F":
  gender_list = [1]
elif gender == "M":
  gender_list = [0]

input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

# 継続か退会かを予測(0は継続、1は退会)
print(model.predict([input_data]))
# 継続か退会それぞれの予測確率を表示
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]


c:\Users\nao23\Documents\Python_Projects\01_sample_100knocks\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\nao23\Documents\Python_Projects\01_sample_100knocks\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
